In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 27 10:35:16 2019

@author: Valentine Z

You agree to the rules and regulations of API usage for NationStates.
As much as I try my best to follow the API rules of NationStates, do be careful about exceeding request times.
Max request rate is 50 req / 30 sec (1.66667 Hz).

My rate-calling calculation is rather primitive, taking in account of the API calls over time (excluding processing in CSV),
but it could be higher or lower. Please do consider this with discretion going forward.

And make VERY sure that using_nation is YOUR nation.
Like my card extractor, you can find out the public data of other nations,
BUT the API call must indicate that it is you.

TODO: Data is raw and will need small touches on Excel - removing quotation marks, adding Adjusted Economic Freedom, etc.
I would recommend that you use the data I provided; this is more of me posting this code so that those who want to can do it at home. ^^
"""

from time import sleep
import time

import requests
import xml.etree.ElementTree as ElementTree
import re
import asyncio
import os
import pandas as pd
import numpy as np

In [ ]:
# User who is going to use this session. Also generates time for use with file names.
# Rerun this to prevent file overwriting, or a change of username.

using_nation = "Valentine Z"
delay = 0.250 # Increase this if it was going too fast.

secondsSinceEpoch = time.time()
timeObj = time.localtime(secondsSinceEpoch)

current_time = '%02d-%02d-%d %02d%02d%02d' % (
timeObj.tm_year, timeObj.tm_mon, timeObj.tm_mday, timeObj.tm_hour, timeObj.tm_min, timeObj.tm_sec)

print("Current TimeStamp for the file from this point is: " + current_time)

In [ ]:
# Get specific region's data.
def get_region_nations(region):

    url = "https://www.nationstates.net/cgi-bin/api.cgi?region=" + region.replace(" ", "_") + "&q=nations"
    r = requests.get(url, headers = headers)
    root = ElementTree.fromstring(r.content)
    
    nation_list = root.find('NATIONS').text
    
    return nation_list

# Get it for every single nation. This is the default.
def get_every_nation():

    url = "https://www.nationstates.net/cgi-bin/api.cgi?q=nations"
    r = requests.get(url, headers = headers)
    root = ElementTree.fromstring(r.content)
    
    nation_list = root.find('NATIONS').text
    
    return nation_list


list_of_expenditures = [
    "ADMINISTRATION",
    "DEFENCE",
    "EDUCATION",
    "ENVIRONMENT",
    "HEALTHCARE",
    "COMMERCE",
    "INTERNATIONALAID",
    "LAWANDORDER",
    "PUBLICTRANSPORT",
    "SOCIALEQUALITY",
    "SPIRITUALITY",
    "WELFARE"
]

list_of_expenditures_lower = [
    "Percentage Government Expenditure: Administration",
    "Percentage Government Expenditure: Defence",
    "Percentage Government Expenditure: Education",
    "Percentage Government Expenditure: Environment",
    "Percentage Government Expenditure: Healthcare",
    "Percentage Government Expenditure: Commerce",
    "Percentage Government Expenditure: International Aid",
    "Percentage Government Expenditure: Law and Order",
    "Percentage Government Expenditure: Public Transport",
    "Percentage Government Expenditure: Social Equality",
    "Percentage Government Expenditure: Spirituality",
    "Percentage Government Expenditure: Welfare"
]

list_of_sectors = [
    "BLACKMARKET",
    "GOVERNMENT",
    "INDUSTRY",
    "PUBLIC"
]

list_of_sectors_lower = [
    "Percentage Market Sector: Black Market",
    "Percentage Market Sector: Government",
    "Percentage Market Sector: Industry",
    "Percentage Market Sector: Public"
]

list_of_deaths_header = [
    "Percentage Death Type: Accident",
    "Percentage Death Type: Acts of God",
    "Percentage Death Type: Animal Attack",
    "Percentage Death Type: Bungee Jumping",
    "Percentage Death Type: Capital Punishment",
    "Percentage Death Type: Cancer",
    "Percentage Death Type: Disappearance",
    "Percentage Death Type: Exposure",
    "Percentage Death Type: Heart Disease",
    "Percentage Death Type: Involuntary Euthanasia",
    "Percentage Death Type: Lost in Wilderness",
    "Percentage Death Type: Malnourishment",
    "Percentage Death Type: Murder",
    "Percentage Death Type: Nuclear Spill",
    "Percentage Death Type: Old Age",
    "Percentage Death Type: Ritual Sacrifice",
    "Percentage Death Type: Scurvy",
    "Percentage Death Type: Space Shuttle Mishap",
    "Percentage Death Type: Suicide While in Police Custody",
    "Percentage Death Type: Sunburn",
    "Percentage Death Type: Vat Leakage",
    "Percentage Death Type: War",
    "Percentage Death Type: Work"
]

list_of_deaths = [
    "Accident",
    "Acts of God",
    "Animal Attack",
    "Bungee Jumping",
    "Capital Punishment",
    "Cancer",
    "Disappearance",
    "Exposure",
    "Heart Disease",
    "Involuntary Euthanasia",
    "Lost in Wilderness",
    "Malnourishment",
    "Murder",
    "Nuclear Spill",
    "Old Age",
    "Ritual Sacrifice",
    "Scurvy",
    "Space Shuttle Mishap",
    "Suicide While in Police Custody",
    "Sunburn",
    "Vat Leakage",
    "War",
    "Work"
]

list_of_stats = ["civil rights",
"civil rights rank",
"economy",
"economy rank",
"political freedoms",
"political freedoms rank",
"population",
"population rank",
"wealth gaps",
"wealth gaps rank",
"death rate",
"death rate rank",
"compassion",
"compassion rank",
"eco-friendliness",
"eco-friendliness rank",
"social conservatism",
"social conservatism rank",
"nudity",
"nudity rank",
"industry: automobile manufacturing",
"industry: automobile manufacturing rank",
"industry: cheese exports",
"industry: cheese exports rank",
"industry: basket weaving",
"industry: basket weaving rank",
"industry: informtion technology",
"industry: informtion technology rank",
"industry: pizza delivery",
"industry: pizza delivery rank",
"industry: trout fishing",
"industry: trout fishing rank",
"industry: arms manufacturing",
"industry: arms manufacturing rank",
"sector: agriculture",
"sector: agriculture rank",
"industry: beverage sales",
"industry: beverage sales rank",
"industry: timber woodchipping",
"industry: timber woodchipping rank",
"industry: mining",
"industry: mining rank",
"industry: insurance",
"industry: insurance rank",
"industry: furniture restoration",
"industry: furniture restoration rank",
"industry: retail",
"industry: retail rank",
"industry: book publishing",
"industry: book publishing rank",
"industry: gambling",
"industry: gambling rank",
"sector: manufacturing",
"sector: manufacturing rank",
"government size",
"government size rank",
"welfare",
"welfare rank",
"public healthcare",
"public healthcare rank",
"law enforcement",
"law enforcement rank",
"business subsidization",
"business subsidization rank",
"religiousness",
"religiousness rank",
"income equality",
"income equality rank",
"niceness",
"niceness rank",
"rudeness",
"rudeness rank",
"intelligence",
"intelligence rank",
"ignorance",
"ignorance rank",
"political apathy",
"political apathy rank",
"health",
"health rank",
"cheerfulness",
"cheerfulness rank",
"weather",
"weather rank",
"compliance",
"compliance rank",
"safety",
"safety rank",
"lifespan",
"lifespan rank",
"ideological radicality",
"ideological radicality rank",
"defense forces",
"defense forces rank",
"pacifism",
"pacifism rank",
"economic freedom",
"economic freedom rank",
"taxation",
"taxation rank",
"freedom from taxation",
"freedom from taxation rank",
"corruption",
"corruption rank",
"integrity",
"integrity rank",
"authoritarianism",
"authoritarianism rank",
"youth rebelliousness",
"youth rebelliousness rank",
"culture",
"culture rank",
"employment",
"employment rank",
"public transport",
"public transport rank",
"tourism",
"tourism rank",
"weaponization",
"weaponization rank",
"recreational drug use",
"recreational drug use rank",
"obesity",
"obesity rank",
"secularism",
"secularism rank",
"environmental beauty",
"environmental beauty rank",
"charmlessness",
"charmlessness rank",
"influence",
"influence rank",
"world assembly endorsements",
"world assembly endorsements rank",
"averageness",
"averageness rank",
"human development index",
"human development index rank",
"primitiveness",
"primitiveness rank",
"scientific advancement",
"scientific advancement rank",
"inclusiveness",
"inclusiveness rank",
"average income",
"average income rank",
"average income of poor",
"average income of poor rank",
"average income of rich",
"average income of rich rank",
"public education",
"public education rank",
"economic output",
"economic output rank",
"crime",
"crime rank",
"foreign aid",
"foreign aid rank",
"black market",
"black market rank",
"residency",
"residency rank",
"survivors",
"survivors rank",
"zombies",
"zombies rank",
"dead",
"dead rank",
"percentage zombies",
"percentage zombies rank",
"average disposable income",
"average disposable income rank",
"deck value",
"deck value rank",
"patriotism",
"patriotism rank",
"food quality",
"food quality rank"]

list_of_hdi_precise = ["hdi-economy", "hdi-smart", "hdi-health", "hdi-precise"]

# Initiate headers

def initiate_headers():
    
    data = []
    
    # data.append(str("Null"))
    
    data.append(str("Nation Name"))
    data.append(str("Category"))
    data.append(str("Number of Issues Answered"))
    data.append(str("Region Name"))
    data.append(str("Flag"))
    
    for i in range(len(list_of_expenditures_lower)):
        data.append(str(list_of_expenditures_lower[i]))
        
    for i in range(len(list_of_sectors_lower)):
        data.append(str(list_of_sectors_lower[i]))
        
    for i in range(len(list_of_deaths_header)):
        data.append(str(list_of_deaths_header[i]))
        
    for i in range(len(list_of_stats)):
        data.append(str(list_of_stats[i]))
        
    for i in range(len(list_of_hdi_precise)):
        data.append(str(list_of_hdi_precise[i]))
        
    data.append(str("Null"))
        
    return str(data) + "\n"

def stat(name):
    
    url = "https://www.nationstates.net/cgi-bin/api.cgi?nation=" + name.replace(" ", "_") + "&q=category+flag+region+census+sectors+deaths+govt+hdi+answered;scale=all"
    r = requests.get(url, headers = headers)
    root = ElementTree.fromstring(r.content)
    
    data = []
    
    data.append(str("Null"))
    data.append(str(name))
    
    # ---------------------------------------------
    # For Category
    
    try:
        category = root.find('CATEGORY').text
        data.append(category)

    except:
        category = "Null.."
        data.append(category)
        
    # ---------------------------------------------
    # For Issues Answered
    
    try:
        answered = root.find('ISSUES_ANSWERED').text
        data.append(answered)

    except:
        answered = "Null.."
        data.append(answered)
        
    # ---------------------------------------------
    # For Region
    
    try:
        region = root.find('REGION').text
        data.append(region)

    except:
        region = "Null.."
        data.append(region)
        
    # ---------------------------------------------
    # For Flag
    
    try:
        flag = root.find('FLAG').text
        data.append(flag)

    except:
        flag = "Null.."
        data.append(flag)
    
    # ---------------------------------------------
    # For Government Expenditure Sector
    
    for expenditure in list_of_expenditures:
        for scale in root.findall('GOVT' + "/" + str(expenditure)):
            try:
                score = scale.text
                data.append(float(score))

            except:
                score = 0
                data.append(float(score))
    # ---------------------------------------------
    
    # For Industry Sector

    for sector in list_of_sectors:
        for scale in root.findall('SECTORS' + "/" + str(sector)):
            try:
                score = scale.text
                data.append(float(score))

            except:
                score = 0
                data.append(float(score))
                
    # ---------------------------------------------
    # For People Sector - Causes of Death
    
    list_of_deaths_for_nation = []

    for scale in root.findall('DEATHS/CAUSE'):
        list_of_deaths_for_nation.append(str(scale.get('type')))

    for element in list_of_deaths:
        if element in list_of_deaths_for_nation:
            scale_index = list_of_deaths_for_nation.index(str(element))
            scale = root.findall('DEATHS/CAUSE')[scale_index]
            score = scale.text
            data.append(float(score))

        else:
            score = 0
            data.append(float(score))
    # ---------------------------------------------
    
    # For Census Sector - NS Stats
    
    for scale in root.findall('CENSUS/SCALE'):
        try:
            id = scale.get('id')
            score = scale.find('SCORE').text
            rank = scale.find('RANK').text
            data.append(float(score))
            data.append(float(rank))
            
        except:
            id = scale.get('id')
            score = scale.find('SCORE').text
            data.append(float(score))
        
    
    # ---------------------------------------------
    # For Precise HDI values.
    
    try:
        hdi_economy = root.find('HDI-ECONOMY').text
        hdi_smart = root.find('HDI-SMART').text
        hdi_lifespan = root.find('HDI-LIFESPAN').text
        hdi_hdi = root.find('HDI').text
        
        data.append(float(hdi_economy))
        data.append(float(hdi_smart))
        data.append(float(hdi_lifespan))
        data.append(float(hdi_hdi))

    except:
        pass
        
    # ---------------------------------------------   
    
    data.append(str("Null"))
    return str(data) + "\n"

def error_placeholder(name):
    
    data = []
    
    data.append(str("Null"))
    data.append(str(name))
    
    data.append(str("Null"))
    
    return str(data) + "\n"

In [ ]:
# For Region Lists. You must have a Regions.txt in the data file. A sample is provided which can be edited.
# You can skip this section if you don't need specific regions.

region_list = open("data/Region.txt", "r")
region_lines = region_list.read()
region_lines = re.split(',|\n|:', region_lines)
headers = {'User-Agent': "Valentine Z's NS Stats and Government Expenditure Retriever, in use by " + using_nation,}

# Check if the website is online / working.
# import urllib

# def check_status():
#     return urllib.request.urlopen("https://www.nationstates.net").getcode()
# # -------

start_time = time.time()

incomplete = []

with open(r'data/Region - ' + str(current_time) + '.csv', 'w+') as file:
    file.write(initiate_headers())
    
    i = 0 # Region Count
    j = 0 # Nation Count
    total = 0
    
    for region in region_lines:
        try:
            region_nation_list = re.split(',|\n|:', get_region_nations(str(region)).strip('"\''))
            
            for nation in region_nation_list:
                try:
                    file.write(stat(nation))
                    j += 1
                    total += 1
                    print(j,"of",len(region_nation_list),"nations done. ",i,"of",len(region_lines),"regions done.")
                    sleep(0.200)
                    
                except KeyboardInterrupt:
                    duration = time.time() - start_time
                    print("--- %s seconds ---" % (duration))
                    print("Total number of API calls: %" % total)
                    print("Rate: ",2 * (total / duration))
                    print("Legal Rate: ",str(1.666667))
                    raise
                    
                except:
                    file.write(error_placeholder(nation))
                    incomplete.append(str(nation))
                    j += 1
                    total += 1
                    print(j,"of",len(region_nation_list),"nations done. ",i,"of",len(region_lines),"regions done.",nation,"does not exist. Continuing.")

            i += 1
            total += 1
            print(i,"of",len(region_lines),"regions done. Region",region,"is done. Continuing...\n")
            

        except KeyboardInterrupt:
            duration = time.time() - start_time
            print("--- %s seconds ---" % (duration))
            print("Total number of API calls: ", total)
            print("Rate: ",2 * (total / duration))
            print("Legal Rate: ",str(1.666667))
            raise

        except:
            i += 1
            total += 1
            print(i,"of",len(region_lines),"regions done. Region",region,"is done. Continuing...\n")
            
        j = 0
        
duration = time.time() - start_time
print("--- %s seconds ---" % (duration))
print("Total number of nation API calls: ", total - i)
print("Total number of region API calls: ", i)
print("Rate: ",2 * (total / duration))
print("Legal Rate: ",str(1.666667))

In [ ]:
# For Nation Lists, when you are finding out for specific nations.
# Again, feel free to skip this if you don't want to use it.

nation_list = open("data/Nations.txt", "r")
nation_lines = nation_list.read()
nation_lines = re.split(',|\n|:', nation_lines)
headers = {'User-Agent': "Valentine Z's NS Stats and Government Expenditure Retriever, in use by " + using_nation,}

# Check if the website is online / working.
# import urllib

# def check_status():
#     return urllib.request.urlopen("https://www.nationstates.net").getcode()
# # -------

start_time = time.time()

incomplete = []

with open(r'data/Nations - ' + str(current_time) + '.csv', 'w+') as file:
    file.write(initiate_headers())
    
    i = 0 # Nation Count

    for nation in nation_lines:
        try:
            file.write(stat(nation))
            i += 1
            print(i,"of",len(nation_lines),"nations done.")
            sleep(delay)
                    
        except KeyboardInterrupt:
            duration = time.time() - start_time
            print("--- %s seconds ---" % (duration))
            print("Total number of API calls: ", i)
            print("Rate: ",2 * (i / duration))
            print("Legal Rate: ",str(1.666667))
            raise
                    
        except:
            file.write(error_placeholder(nation))
            incomplete.append(str(nation))
            i += 1
            print(i,"of",len(nation_lines),"nations done. ",nation,"does not exist. Continuing.")

        
duration = time.time() - start_time
print("--- %s seconds ---" % (duration))
print("Total number of nation API calls: ", i)
print("Rate: ",2 * (i / duration))
print("Legal Rate: ",str(1.666667))

In [ ]:
# For Nation Lists - World
# This is the DEFAULT setting, extracing every single nation. This takes around 3-5 days.

using_nation = "Valentine Z"
headers = {'User-Agent': "Valentine Z's NS Stats and Government Expenditure Retriever, in use by " + using_nation,}

world_nation_list = re.split(',|\n|:', get_every_nation().strip('"\''))

start_time = time.time()

incomplete = []

with open(r'data/All Nations - ' + str(current_time) + '.csv', 'w+') as file:
    file.write(initiate_headers())
    
    i = 0 # Nation Count
    
    for nation in world_nation_list:
        try:
            file.write(stat(nation))
            i += 1
            print(i,"of",len(world_nation_list),"nations done.")
            sleep(delay)
                    
        except KeyboardInterrupt:
            duration = time.time() - start_time
            print("--- %s seconds ---" % (duration))
            print("Total number of API calls: ", i)
            print("Rate: ",2 * (i / duration))
            print("Legal Rate: ",str(1.666667))
            raise
                    
        except:
            file.write(error_placeholder(nation))
            incomplete.append(str(nation))
            i += 1
            print(i,"of",len(world_nation_list),"nations done. ",nation,"does not exist. Continuing.")

        
duration = time.time() - start_time
print("--- %s seconds ---" % (duration))
print("Total number of nation API calls: ", i)
print("Rate: ",2 * (i / duration))
print("Legal Rate: ",str(1.666667))

In [ ]:
# Write down list of incomplete nations in a separate file to be run.

with open(r'Incomplete Nations' + str(current_time) + '.txt', 'w+') as file:
    file.write(str(incomplete))

In [ ]:
# Reading data and processing as dataframe. The master dataframe is termed df1.

# To clean up and format the data. There were many hoops to this because when I used to do this on Excel, it has the tendency to
# mess up the formatting for numbers.
folder = "data"
file_name = "All Nations - 2021-06-15 194113.csv"
file_path = os.path.join(folder, file_name)

df_process = pd.read_csv(file_path, encoding = 'latin-1')

pd.set_option('display.max_columns', 500) # Show all the columns (max 500, but we will have maybe 400 col max)

df_process = df_process.replace({' \'': ''}, regex = True)
df_process = df_process.replace({'\'': ''}, regex = True)
df_process = df_process.replace({'\[': ''}, regex = True)
df_process = df_process.replace({'\]': ''}, regex = True)

df_process = df_process.iloc[:, :-1]
df_process.rename(columns = lambda x: x.replace(' \'', ''), inplace = True)
df_process.rename(columns = lambda x: x.replace('\'', ''), inplace = True)
df_process.rename(columns = lambda x: x.replace('[', ''), inplace = True)
df_process.rename(columns = lambda x: x.replace(']', ''), inplace = True)
df_process.rename(columns = lambda x: x.replace('informtion', 'information'), inplace = True) # Damn typo.

df_process = df_process.reset_index().iloc[:, 1:]

df_process.loc[df_process['economic freedom'] < 0, "adjusted economic freedom"] = df_process["economic freedom"] + 100
df_process.loc[df_process['economic freedom'] > 0, "adjusted economic freedom"] = df_process["economic freedom"]
df_process.loc[df_process['economic freedom'] == 0, "adjusted economic freedom"] = 50

df_process = df_process.dropna(how = 'any')

df_process.to_csv("data/" + file_name[:-4] + "(Processed).csv")